In [ ]:
#importing required libraries
import numpy as np
import nltk
import string
import random
import requests

In [ ]:
#importing and reading corpus
f=open('ronabot.txt','r',errors = 'ignore')
raw_doc=f.read()
raw_doc=raw_doc.lower()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
sent_tokens = nltk.sent_tokenize(raw_doc)
word_tokens = nltk.word_tokenize(raw_doc)


In [ ]:
#sentence token example
sent_tokens[:2]

In [ ]:
#word tokens example
word_tokens[:2]

In [ ]:
#Text preprocessing
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically oriented dictionary of english included in NLTK
stopwords = nltk.corpus.stopwords
def LemTokens(tokens):
  return [lemmer.lemmatize(token, 'v') for token in tokens if token not in set(stopwords.words('english'))]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
#Defining the Greeting Function
GREET_INPUTS = ("hello","hi","greetings","sup","whats up","hey")
GREET_RESPONSES = ["hi","hey","*nods*","hi there","hello"]
def greet(sentence):

  for word in sentence.split():
    if word.lower() in GREET_INPUTS:
      return random.choice(GREET_RESPONSES)

In [ ]:
#genrating responses
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def response(user_response):
  robo1_response=''
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, analyzer='word', max_features=80)
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf)==0:
    robo1_response = robo1_response + "I am sorry, I don't understand you"
    return robo1_response
  else:
    robo1_response = robo1_response + sent_tokens[idx]
    return robo1_response

In [ ]:
#Defining conversation start and end protocols
flag=True
print('''RonaBot: Welcome! You can ask me anything about COVID-19 in Canada as at April 20,2022.
To exit at any point, type bye''')
while(flag == True):
  user_response = input('User: ')
  user_response = user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag = False
      print('RonaBot: You are welcome')
    else:
      if(greet(user_response)!= None):
        print("RonaBot: "+greet(user_response))
      else:
        sent_tokens.append(user_response)
        word_tokens = word_tokens+ nltk.word_tokenize(user_response)
        final_words = list(set(word_tokens))
        print("RonaBot: ", end="")
        print(response(user_response))
        sent_tokens.remove(user_response)
  else:
    flag=False
    print("RonaBot: Goodbye! Stay Safe..")
